In [1]:
import os.path as osp
import sys
import json

sys.path.append(osp.abspath('..'))

from utils.mdata import load_pkl
from utils.face_landmark import FaceAlignmentDetector

hndata_root = '/mnt/c/Users/anton/Research/3DHeadGen/RecolorNeRF/sample/Ours/HeadNeRF/1024x1024/0001/00'
hncams_file = osp.join(hndata_root, 'cams.pkl')
hncams_data = load_pkl(hncams_file)
phdata_root = '/mnt/c/Users/anton/Research/3DHeadGen/RecolorNeRF/sample/Ours/HeadNeRF/1024x1024/0001/00PanoHead'
phjson_file = osp.join(phdata_root, 'meta_1-1.json')
with open(phjson_file, 'r') as f:
    phjson_data = json.load(f)
flmk_det = FaceAlignmentDetector()

In [39]:
import cv2
import numpy as np
import tqdm

def cal_ray(inmat, c2w, pt):
    fx = inmat[0, 0]
    fy = inmat[1, 1]
    cx = inmat[0, 2]
    cy = inmat[1, 2]
    u, v = pt
    direction = [(u - cx + 0.5) / fx, (v - cy + 0.5) / fy, 1.]
    ray_o = c2w[:3, 3]
    ray_d = direction @ c2w[:3, :3].T
    ray_d = ray_d / np.linalg.norm(ray_d)
    return ray_o, ray_d

hn_rays_o, hn_rays_d = [], []
ph_rays_o, ph_rays_d = [], []
for k in tqdm.tqdm(sorted(phjson_data.keys())):
    image_name = k.split('/')[1].replace('.png', '')

    hnimg_data = cv2.imread(osp.join(hndata_root, 'image', f'{image_name}.png'))
    hnimg_lmks = flmk_det(hnimg_data, True)
    
    phimg_data = cv2.imread(osp.join(phdata_root, 'align_images', f'{image_name}_00.png'))
    phimg_lmks = flmk_det(phimg_data, True)

    hnimg_inmat = hncams_data['nl3dmm'][f'{image_name}.png'].inmat
    hnimg_c2w = hncams_data['nl3dmm'][f'{image_name}.png'].c2w.tfmat
    hnimg_rays_o, hnimg_rays_d = [], []
    for i in range(len(hnimg_lmks)):
        ray_o, ray_d = cal_ray(hnimg_inmat, hnimg_c2w, hnimg_lmks[i])
        hnimg_rays_o.append(ray_o)
        hnimg_rays_d.append(ray_d)
    hnimg_rays_o = np.array(hnimg_rays_o)
    hn_rays_o.append(hnimg_rays_o)
    hnimg_rays_d = np.array(hnimg_rays_d)
    hn_rays_d.append(hnimg_rays_d)
    
    phimg_cam =  np.array(phjson_data[f'image/{image_name}.png']['head']['00']['camera'])
    phimg_c2w = phimg_cam[:16].reshape(4, 4)
    phimg_inmat = phimg_cam[16:].reshape(3, 3)
    phimg_inmat[:2, :] *= 563
    phimg_rays_o, phimg_rays_d = [], []
    for i in range(len(phimg_lmks)):
        ray_o, ray_d = cal_ray(phimg_inmat, phimg_c2w, phimg_lmks[i])
        phimg_rays_o.append(ray_o)
        phimg_rays_d.append(ray_d)
    phimg_rays_o = np.array(phimg_rays_o)
    ph_rays_o.append(phimg_rays_o)
    phimg_rays_d = np.array(phimg_rays_d)
    ph_rays_d.append(phimg_rays_d)
hn_rays_o = np.array(hn_rays_o)
hn_rays_d = np.array(hn_rays_d)
ph_rays_o = np.array(ph_rays_o)
ph_rays_d = np.array(ph_rays_d)

In [44]:
def calculate_nearest_point_of_rays(rays_o: np.ndarray, rays_d: np.ndarray):
    # rays_o, rays_d: [N, 3, 1], sum(rays_d ** 2) == 1
    A_i = np.eye(3) - rays_d * np.transpose(rays_d, [0, 2, 1])
    b_i = -A_i @ rays_o
    pt_mindist = np.squeeze(-np.linalg.inv((np.transpose(A_i, [0, 2, 1]) @ A_i).mean(0)) @ (b_i).mean(0))
    return pt_mindist


hn_lmks_3d = []
for i in range(68):
    center3D = calculate_nearest_point_of_rays(hn_rays_o[:, i, :, None], hn_rays_d[:, i, :, None])
    hn_lmks_3d.append(center3D)
hn_lmks_3d = np.array(hn_lmks_3d)

ph_lmks_3d = []
for i in range(68):
    center3D = calculate_nearest_point_of_rays(ph_rays_o[:, i, :, None], ph_rays_d[:, i, :, None])
    ph_lmks_3d.append(center3D)
ph_lmks_3d = np.array(ph_lmks_3d)

In [61]:
import utils.visualize as uvis
import importlib
importlib.reload(uvis)
from utils.visualize import MMeshMeta

m = MMeshMeta()
m.create_pointcloud(hn_lmks_3d, np.array([[0, 0, 255]]).repeat(hn_lmks_3d.shape[0], 0).astype(np.uint8))
m.save('hn_lmks_3d.ply')

m = MMeshMeta()
m.create_pointcloud(ph_lmks_3d, np.array([[0, 0, 255]]).repeat(ph_lmks_3d.shape[0], 0).astype(np.uint8))
m.save('ph_lmks_3d.ply')

m1, m2 = MMeshMeta(), MMeshMeta()
for k in sorted(phjson_data.keys()):
    image_name = k.split('/')[1].replace('.png', '')

    hnimg_inmat = hncams_data['nl3dmm'][f'{image_name}.png'].inmat
    hnimg_c2w = hncams_data['nl3dmm'][f'{image_name}.png'].c2w.tfmat
    m1.create_camera_coordinates(hnimg_c2w, length=1., eps=1.0e-2)
    
    phimg_cam =  np.array(phjson_data[f'image/{image_name}.png']['head']['00']['camera'])
    phimg_c2w = phimg_cam[:16].reshape(4, 4)
    phimg_inmat = phimg_cam[16:].reshape(3, 3)
    phimg_inmat[:2, :] *= 563
    m2.create_camera_coordinates(phimg_c2w, length=1., eps=1.0e-2)
m1.save('hn_cams.ply')
m2.save('ph_cams.ply')


True

In [62]:
from utils import msimil
def simil_analysis(X0, X1):
    # m, r, t = msimil.process(source_points, target_points)
    m, r, t = msimil.process(X1, X0)
    print(f'm:\n{m}\nr: {r}\nt:\n{t}')
    T1to0 = np.eye(4)
    T1to0[:3, :3] = m * r
    T1to0[:3, 3:] = t
    print(f'T1to0:\n{T1to0}')
    return T1to0

Tfmatrix = simil_analysis(ph_lmks_3d, hn_lmks_3d)  # from input to ref

m:
0.23101691783912445
r: [[ 0.99870387  0.02345814 -0.0451696 ]
 [-0.02672519  0.99696366 -0.07313846]
 [ 0.04331675  0.07425083  0.99629839]]
t:
[[ 0.00049117]
 [-0.0314007 ]
 [ 0.08547127]]
T1to0:
[[ 2.30717490e-01  5.41922775e-03 -1.04349408e-02  4.91167526e-04]
 [-6.17397087e-03  2.30315471e-01 -1.68962217e-02 -3.14006961e-02]
 [ 1.00069028e-02  1.71531978e-02  2.30161782e-01  8.54712734e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [65]:
def transform_points(pts, tfmatrix):
    if pts.shape[1] == 3:
        pts_ = np.concatenate([pts, np.ones((pts.shape[0], 1))], axis=1)
    elif pts.shape[1] == 4:
        pts_ = pts
    else:
        raise ValueError(f'pts.shape[1] must be 3 or 4, but got {pts.shape[1]}')
    tfpts = (tfmatrix @ pts_.T).T[:, :3]
    return tfpts


def transform_cam(T_c2w, tfmatrix):
    T_c2w_ = tfmatrix @ T_c2w
    return T_c2w_


hn_lmks_3d_tf = transform_points(hn_lmks_3d, Tfmatrix)

m = MMeshMeta()
m.create_pointcloud(hn_lmks_3d_tf, np.array([[0, 0, 255]]).repeat(hn_lmks_3d_tf.shape[0], 0).astype(np.uint8))
m.save('hn_lmks_3d_tf.ply')

m= MMeshMeta()
for k in sorted(phjson_data.keys()):
    image_name = k.split('/')[1].replace('.png', '')

    hnimg_inmat = hncams_data['nl3dmm'][f'{image_name}.png'].inmat
    hnimg_c2w = hncams_data['nl3dmm'][f'{image_name}.png'].c2w.tfmat
    hnimg_c2w_tf = transform_cam(hnimg_c2w, Tfmatrix)
    m.create_camera_coordinates(hnimg_c2w_tf, length=1., eps=1.0e-2)
m.save('hn_cams_tf.ply')

True

In [66]:
hn2ph_tfmat = Tfmatrix

In [68]:
np.save('hn2ph_tfmat.npy', hn2ph_tfmat)